# Canopy simulation tutorial

This notebook contains sample cases for the Eradiate preview session of Dec 7th 2021. In this tutorial, we introduce the canopy simulation capabilities of Eradiate. We will explore the features and interfaces of different objects used to construct a scene, simulate radiative transfer and visualise results.

In [ ]:
# We load the Rich notebook extension for improved object inspection
%load_ext rich

# We import general processing and plotting libraries
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import pandas as pd

# We import the top-level Eradiate module
import eradiate

# For convenience, we alias a few components
from eradiate import unit_registry as ureg
import eradiate.scenes as ertsc
import eradiate.experiments as ertxp

# We'll use the correlated k-distribution mode in double precision
eradiate.set_mode("ckd_double")

# We activate a few convenient presets
eradiate.notebook.install()
eradiate.plot.set_style(rc={"figure.dpi": 96})
eradiate.config.progress = 1  # Restrict progress display to the highest level

We start with the definition of a couple of utility functions which will simplify the visualisation of results.

In [ ]:
def show_camera(exp, measure_id):
    """
    Display the output of a monochromatic camera measure.
    """
    fig, ax = plt.subplots(1,1)
    exp.results[measure_id]["radiance"].squeeze(drop=True).plot.imshow(
        ax=ax, origin="upper", robust=True, cmap="Greys_r"
    )
    ax.set_aspect(1)
    plt.show()
    plt.close()

def show_brf(exp, measure_id):
    """
    Display the output of a 
    """
    fig, ax = plt.subplots(1,1)
    exp.results[measure_id]["brf"].squeeze(drop=True).plot(
        ax=ax, ls=":", marker="."
    )
    plt.show()
    plt.close()

## Surface only

First, let's create a very simple scene consisting of a default Lambertian surface observed by a camera.

In [ ]:
camera_oblique = eradiate.scenes.measure.PerspectiveCameraMeasure(
    id="camera_oblique",
    origin=[15, 15, 15] * ureg.m,
    target=[0, 0, 0],
    up=[0, 0, 1],
    film_resolution=(320, 240),
    spp=512,
)

exp = eradiate.experiments.RamiExperiment(
    surface=eradiate.scenes.surface.LambertianSurface(width=10 * ureg.m),
    measures=[camera_oblique],
)

In [ ]:
exp.run()
show_camera(exp, "camera_oblique")

## Surface + canopy

Let's add canopy above this ground patch. We'll configure a homogeneous canopy composed of randomly oriented floating disks.

In [ ]:
homogeneous_canopy = eradiate.scenes.biosphere.DiscreteCanopy.homogeneous(
    l_vertical=1.0 * ureg.m,
    l_horizontal=10.0 * ureg.m,
    lai=2.0,
    leaf_radius=10 * ureg.cm,
)

In [ ]:
exp = eradiate.experiments.RamiExperiment(
    surface=eradiate.scenes.surface.LambertianSurface(),
    canopy=homogeneous_canopy,
    measures=[camera_oblique],
)
exp.run()
show_camera(exp, "camera_oblique")


## Surface + padded canopy

In order to simulate the fact that our canopy is periodic, we can pad it with clones of itself. Let's start with just of few of them.


In [ ]:
exp = eradiate.experiments.RamiExperiment(
    surface=eradiate.scenes.surface.LambertianSurface(),
    canopy=homogeneous_canopy,
    padding=1,
    measures=[camera_oblique],
)
exp.run()
show_camera(exp, "camera_oblique")

Now, let's relocate our camera to the top of canopy.

In [ ]:
camera_toc = eradiate.scenes.measure.PerspectiveCameraMeasure(
    id="camera_toc",
    origin=[15, 15, 2] * ureg.m,
    target=[0, 0, 0],
    up=[0, 0, 1],
    film_resolution=(320, 240),
    spp=512,
)
exp = eradiate.experiments.RamiExperiment(
    surface=eradiate.scenes.surface.LambertianSurface(),
    canopy=homogeneous_canopy,
    padding=2,
    measures=[camera_toc],
)
exp.run()
show_camera(exp, "camera_toc")

## BRF in principal plane

Now that we know how to build our scene, we are going to compute the BRF in the principal plane for a similar scene. This is similar to the test cases for which we can submit results to the RAMI online checker (ROMC.)

In order to observe something meaningful, let's set the Sun zenith angle to 30°.

In [ ]:
illumination = eradiate.scenes.illumination.DirectionalIllumination(
    zenith=30.0, azimuth=0.0
)

brf_pplane = eradiate.scenes.measure.MultiDistantMeasure.from_viewing_angles(
    id="brf_pplane",
    azimuths=0.0,
    zeniths=np.arange(-75, 76, 5),
    spp=50000,
)

exp = eradiate.experiments.RamiExperiment(
    illumination=illumination,
    surface=eradiate.scenes.surface.LambertianSurface(),
    canopy=homogeneous_canopy,
    padding=10,
    measures=[brf_pplane],
)
exp.run()
show_brf(exp, "brf_pplane")

There still is some Monte Carlo noise, but we're keeping the number of samples low in order so that the simulation remains short.